In [1]:
%matplotlib inline 

import pandas as pd
import numpy as np
from sklearn import cluster
from sklearn import metrics
from sklearn.metrics import pairwise_distances
import matplotlib.pyplot as plt
import matplotlib
matplotlib.style.use('ggplot') 

import seaborn as sns

In [2]:
data =  pd.read_csv("home_loans_history_2018-04-02.csv" , encoding = 'latin-1')

In [3]:
data.columns

Index(['collectiondate', 'isdiscontinued', 'uuid', 'variationuuid',
       'createdat', 'updatedat', 'productname', 'variationname',
       'gotositeenabled', 'gotositeurl',
       ...
       'otherrestrictions', 'nswapplicable', 'vicapplicable', 'waapplicable',
       'ntapplicable', 'tasapplicable', 'qldapplicable', 'saapplicable',
       'actapplicable', 'filename'],
      dtype='object', length=107)

In [6]:
data= data[["productname", "variationname","companyname","homeloantype","mintotalloanamount","maxlvr","minlvr",
            "fixmonth","allowssplitloan","hasoffsetaccount","isfirsthomebuyersavailable","requiresfirsthomebuyers",
            "hasowneroccupiedpurpose","hasinvestmentpurpose","hasprincipalandinterest","hasinterestonly","minloanterm",
            "maxloanterm","hasweeklyrepayments","hasfortnightlyrepayments","hasmonthlyrepayments","isnewcustomer"]]
#"maxtotalloanamount"

In [7]:
data["fixmonth"]=data["fixmonth"].astype(str)
data['minloanterm']=data['minloanterm'].astype(str)
data['maxloanterm']=data['maxloanterm'].astype(str)

In [8]:
data.dtypes

productname                    object
variationname                  object
companyname                    object
homeloantype                   object
mintotalloanamount            float64
maxlvr                        float64
minlvr                        float64
fixmonth                       object
allowssplitloan                  bool
hasoffsetaccount                 bool
isfirsthomebuyersavailable       bool
requiresfirsthomebuyers          bool
hasowneroccupiedpurpose          bool
hasinvestmentpurpose             bool
hasprincipalandinterest          bool
hasinterestonly                  bool
minloanterm                    object
maxloanterm                    object
hasweeklyrepayments              bool
hasfortnightlyrepayments         bool
hasmonthlyrepayments             bool
isnewcustomer                    bool
dtype: object

In [9]:
data["maxlvr"].unique()

array([ 85.,  90.,  95.,  70.,  80.,  60.,  75.,  nan,  55.,  65.,  98.,
        97.,  50.,  30.])

# Imputation (Filling missing data)

In [10]:
data['mintotalloanamount']=data['mintotalloanamount'].replace([np.NaN,np.nan],0)
# data['maxtotalloanamount']=data['maxtotalloanamount'].replace([np.NaN,np.nan],2000000)
data['maxlvr']=data['maxlvr'].replace([np.NaN,np.nan],95)
data['minlvr']=data['minlvr'].replace([np.NaN,np.nan],0)
# data['fixmonth']=data['fixmonth'].replace([np.NaN,np.nan],360)

# Gower Distance

In [11]:
from sklearn.neighbors import DistanceMetric


def gower_distance(X):
    """
    This function expects a pandas dataframe as input
    The data frame is to contain the features along the columns. Based on these features a
    distance matrix will be returned which will contain the pairwise gower distance between the rows
    All variables of object type will be treated as nominal variables and the others will be treated as 
    numeric variables.
    Distance metrics used for:
    Nominal variables: Dice distance (https://en.wikipedia.org/wiki/S%C3%B8rensen%E2%80%93Dice_coefficient)
    Numeric variables: Manhattan distance normalized by the range of the variable (https://en.wikipedia.org/wiki/Taxicab_geometry)
    """
    individual_variable_distances = []

    for i in range(X.shape[1]):
        feature = X.iloc[:,[i]]
        if (feature.dtypes[0] == np.object) or (feature.dtypes[0] == np.object):
            feature_dist = DistanceMetric.get_metric('dice').pairwise(pd.get_dummies(feature))
        else:
            feature_dist = DistanceMetric.get_metric('manhattan').pairwise(feature) / np.ptp(feature.values)
            
        individual_variable_distances.append(feature_dist)

    return np.array(individual_variable_distances).mean(0)


In [12]:
X= data.drop(["productname", "variationname","companyname"],axis=1)

In [13]:
df=gower_distance(X) ; df

array([[ 0.        ,  0.05263158,  0.05263158, ...,  0.57696938,
         0.58960096,  0.5379257 ],
       [ 0.05263158,  0.        ,  0.05263158, ...,  0.57696938,
         0.58960096,  0.5379257 ],
       [ 0.05263158,  0.05263158,  0.        , ...,  0.57696938,
         0.58960096,  0.5379257 ],
       ..., 
       [ 0.57696938,  0.57696938,  0.57696938, ...,  0.        ,
         0.01263157,  0.0622635 ],
       [ 0.58960096,  0.58960096,  0.58960096, ...,  0.01263157,
         0.        ,  0.07489508],
       [ 0.5379257 ,  0.5379257 ,  0.5379257 , ...,  0.0622635 ,
         0.07489508,  0.        ]])

In [14]:
from sklearn.preprocessing import normalize

Xn = normalize(df)

# Clustering with K-means

In [15]:
from sklearn.cluster import KMeans

In [16]:
#6 clusters
k_mean6 = KMeans(n_clusters=6)
k_mean6.fit(Xn)

KMeans(algorithm='auto', copy_x=True, init='k-means++', max_iter=300,
    n_clusters=6, n_init=10, n_jobs=1, precompute_distances='auto',
    random_state=None, tol=0.0001, verbose=0)

In [17]:
print(k_mean6)

KMeans(algorithm='auto', copy_x=True, init='k-means++', max_iter=300,
    n_clusters=6, n_init=10, n_jobs=1, precompute_distances='auto',
    random_state=None, tol=0.0001, verbose=0)


In [18]:
# Labels and centroids for 6 Clusters
labels_6 = k_mean6.labels_
print(labels_6)
clusters_6= k_mean6.cluster_centers_
print(clusters_6)


[2 2 2 ..., 1 1 4]
[[ 0.01512142  0.01512665  0.01513726 ...,  0.01301844  0.01343938
   0.01183493]
 [ 0.01875947  0.01875947  0.01875947 ...,  0.00959685  0.0099798
   0.00864819]
 [ 0.00883227  0.00884213  0.00888077 ...,  0.0212623   0.02180402
   0.02036452]
 [ 0.01334584  0.01334584  0.01334584 ...,  0.01548481  0.01587872
   0.01488683]
 [ 0.01805     0.01806408  0.01806676 ...,  0.00994274  0.01038868
   0.00850193]
 [ 0.01372472  0.01374374  0.01377257 ...,  0.01626068  0.01679132
   0.01501745]]


In [19]:
data6 = data
data6['labels'] = labels_6

In [20]:
data6.columns

Index(['productname', 'variationname', 'companyname', 'homeloantype',
       'mintotalloanamount', 'maxlvr', 'minlvr', 'fixmonth', 'allowssplitloan',
       'hasoffsetaccount', 'isfirsthomebuyersavailable',
       'requiresfirsthomebuyers', 'hasowneroccupiedpurpose',
       'hasinvestmentpurpose', 'hasprincipalandinterest', 'hasinterestonly',
       'minloanterm', 'maxloanterm', 'hasweeklyrepayments',
       'hasfortnightlyrepayments', 'hasmonthlyrepayments', 'isnewcustomer',
       'labels'],
      dtype='object')

In [21]:
data6.to_csv("data6.csv",index=None)